<img src="..\img\Logo.png" width=150 align="left" /> <img src="..\img\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>MAPS</font></center>

> # <font color='steelblue'> <b>POIS ON MAP</font>

### <font color='steelblue'>Description</font>

 Dibujar un conjunto n puntos de interés (POIS) sobre un mismo mapa.
 
- Cada una de los POI viene dado por su posición en formato (Lat, Long).
- Capa POI tiene un conjunto de información asociada al mismo en forma de columnas

- Se debe poder seleccionar los POIs que se quieren mostrar, en base a categorías comunes que estos tienen (e.g. museo)
    - Las selecciones son todas de tipo check box (AND) no radio button (OR)
    
- Cada tipología PRINCIPAL de POI seleccionada y que se muestre debe poderse diferenciar con un icono distinto
    - Valorar y poner como opcional (si existe) la opción de indicar una imagen a modo de icono

- Se indicarán las columnas de las categorías por orden de izqueirda a derecha

- En la tarjeta del POI al poner el ratón aparecerá la info de las columnas que se indiquen n los parámetros que serán variables dependiendo del caso

- En la leyenda aparecerá el número de elementos de cada tipo que haya en la columna que se indique


<img src="..\img\POIS.jpg" width=550 align="left" />

## <font color='green'>Settings</font>

In [1]:
# Libraries to use


import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from branca.element import Figure
import folium

import dash
from dash import dcc
from dash import html 
from dash.dependencies import Input, Output
import plotly.express as px
from collections import Counter


## <font color='green'>Data Load</font>

In [2]:
#Load pois
file_name='POIS_MADRID.csv'
pois = pd.read_csv('../data/'+file_name , header=0)[0:500]

## <font color='green'>PARAMETERS</font>

In [3]:
# Categorías
CATEGORIAS=['TIPO1','TIPO2','TIPO3','TIPO4','TIPO5']
sel_category_symbol = "TIPO1" #catgoria de los iconos

categoria_sel_legend = "TIPO1"  # categoria para la leyenda

symbol_dict = { #diccionario con las urls de los iconos para imagenes customizadas
    "Serv" : "https://script-lab.azureedge.net/assets/images/icon-32.png",
    "others" : "https://i.stack.imgur.com/6cDGi.png"
}

# Columnas para la info en la tarjeta del POI
choroplet_info=['NOMBRE','DESCRIPCION','DIRECCION']


Parámetros para configurar el aspecto del mapa

In [4]:
# Titulo del mapa
titulo = 'POIs Madrid'
# Zoom inicial
zoom = 11
# Tamaño del mapa
alto=500
# Tipo de mapa
estilo_mapa='open-street-map' # "open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain",                           
            # "stamen-toner" or "stamen-watercolor" 
    
iconsize = [34, 18]

# 

## <font color='green'>POIs MAP</font>

In [5]:
#Transformaciones sobre los parametros


choroplet_info_dict = {}
for i in pois.to_dict().keys():
    if i in choroplet_info:
        choroplet_info_dict[i] = True
    else:
        choroplet_info_dict[i] = False
        
#Transformaciones sobre los datos

pois['LATITUD'] = pois['LATITUD'].astype(float)
pois['LONGITUD'] = pois['LONGITUD'].astype(float)
pois['symbol'] = pois['TIPO1'].apply(lambda x: symbol_dict.get(x, symbol_dict["others"]))


In [ ]:
import dash
import dash_leaflet as dl
from dash import html
check_elements = []
for i,categoria in enumerate(CATEGORIAS):
    label = html.Label(categoria, style={'color': '#393939', 'font-weight': 'bold'})
    check = dcc.Checklist(
        id='tipo-poi'+str(i),
        options=[{'label': tipo, 'value': tipo} for tipo in pois[categoria].unique()],
        value=pois[categoria].unique().tolist(),
        labelStyle={'display': 'block', 'margin-bottom': '10px', 'color': '#393939'}
    )
    check_elements.append(label)
    check_elements.append(check)

    
app = dash.Dash()
 
    
markers = []
for index, row in pois.iterrows():
    marker = dl.Marker(position=[row['LATITUD'], row['LONGITUD']], 
                       icon={"iconUrl": row["symbol"],"iconSize": iconsize},
                       children= dl.Tooltip(["\n".join(x) for x in pois[choroplet_info].values.tolist()][index]))
    
    markers.append(marker)

legend = []
categoria_sel = Counter(pois[categoria_sel_legend])
for k,i in categoria_sel.items():
    legend.append(html.Label(str(k) + " : " + str(i), style={'color': '#393939', 'font-weight': 'bold'}))
    legend.append(html.Br())
    
  
app.layout = html.Div([
    html.H1(children=titulo),
    html.Div(style={'display': 'inline-block', 'width': '15%', 'vertical-align': 'top', 'max-height': '600px', 'overflow-y': 'scroll', 'background-color': '#E6E6E6', 'padding': '10px'}, children=
    check_elements,
    ),
    dl.Map(id="map",style={'width': '55%', 'height': '500px','display': 'inline-block',}, center=[40.416729,-3.703339], zoom=zoom, children=[        dl.TileLayer(),        *markers    ]),
    html.Div(id="legend",style={'display': 'inline-block', 'width': '15%', 'vertical-align': 'top', 'max-height': '100px'}, children=[*legend])

])

inputs = [Input('tipo-poi' + str(i), 'value') for i in range(len(CATEGORIAS))]

@app.callback(
    Output('map', 'children'),
    inputs
    )
def update_map(*selected_pois):
    total_df = pd.DataFrame(columns=pois.columns)
    for i, poi_type in enumerate(CATEGORIAS):
        filtered_df = pois.copy()
        filtered_df = filtered_df[filtered_df[poi_type].isin(selected_pois[i])]
        total_df = pd.concat([total_df,filtered_df])
    total_df = total_df.drop_duplicates()

    
    filtered_df = total_df
    markers = []
    for index, row in filtered_df.iterrows():
        marker = dl.Marker(position=[row['LATITUD'], row['LONGITUD']], 
                       icon={"iconUrl": row["symbol"], "iconSize": iconsize},
                       children= dl.Tooltip(["\n".join(x) for x in pois[choroplet_info].values.tolist()][index]))
        markers.append(marker)
    return [dl.TileLayer(),*markers]


@app.callback(
    Output('legend', 'children'),
    inputs
    )
def update_legend(*selected_pois):
    total_df = pd.DataFrame(columns=pois.columns)
    for i, poi_type in enumerate(CATEGORIAS):
        filtered_df = pois.copy()
        filtered_df = filtered_df[filtered_df[poi_type].isin(selected_pois[i])]
        total_df = pd.concat([total_df,filtered_df])
    total_df = total_df.drop_duplicates()
    legend = []
    
    categoria_sel = Counter(total_df[categoria_sel_legend])
    for k,i in categoria_sel.items():
        legend.append(html.Label(str(k) + " : " + str(i), style={'color': '#393939', 'font-weight': 'bold'}))
        legend.append(html.Br())
    return legend
    
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
